In [1]:
import os
import gc
from joblib import dump
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

# load files

In [2]:
train = pd.read_parquet("../input/train_base_shifted.parquet")

# define loss and metric

In [3]:
def amex_metric(y_true, y_pred):
    
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    
    for i in [1, 0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
        
    return 0.5 * (gini[1]/gini[0] + top_four)

In [4]:
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return "amex_metric", amex_metric(y_true, y_pred), True

# define training config

In [13]:
seed = 42
n_folds = 5

features = [col for col in train.columns if col not in ["customer_ID", "target"]]
features = [col for col in features if ("D_87" not in col)]

target = "target"
cat_features_base = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
] 
cat_features = [
    "{}_last".format(feature) for feature in cat_features_base
]

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting": "dart",
    "seed": seed,
    "num_leaves": 100,
    "learning_rate": 0.01,
    "feature_fraction": 0.20,
    "bagging_freq": 10,
    "bagging_fraction": 0.50,
    "n_jobs": -1,
    "lambda_l2": 2,
    "min_data_in_leaf": 40
}

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seed_everything(seed)

In [16]:
def training(train):
    
    # create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    
    kfold = StratifiedKFold(
        n_splits=n_folds, 
        shuffle=True, 
        random_state=seed
    )
    
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[target])):
        
        print("#" * 50)
        print("Training fold {} with {} features...".format(fold, len(features)))
        
        x_train, x_val = train.loc[trn_ind, features], train.loc[val_ind, features]
        y_train, y_val = train.loc[trn_ind, target], train.loc[val_ind, target]
        
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)
        model = lgb.train(
            params=params,
            train_set=lgb_train,
            num_boost_round=10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds=100,
            verbose_eval = 500,
            feval = lgb_amex_metric
        )
        # save best model
        dump(model, "../ckpt/lgbm_{}_{}.pkl".format(fold, seed))
        
        # predict validation
        val_pred = model.predict(x_val)
        
        # add to out of folds array
        oof_predictions[val_ind] = val_pred
        
        # compute fold metric
        score = amex_metric(y_val, val_pred)
        print("fold {} score is {}".format(fold, score))
        
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
        
    # compute oof
    score = amex_metric(train[target], oof_predictions)
    print("oof score is {}".format(score))
    
    # create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({"customer_ID": train["customer_ID"], "target": train[target], "prediction": oof_predictions})
    oof_df.to_parquet("lgbm_oof_{}.parquet".format(seed))
    
    return oof_df

In [ ]:
training(train)

##################################################
Training fold 0 with 1578 features...


/data2/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/data2/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/data2/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.722905 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 254142
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 1572


/data2/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/data2/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523


/data2/caozhehan/miniconda3/envs/trade/lib/python3.8/site-packages/lightgbm/callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')
